https://fastapi.tiangolo.com/tutorial/sql-databases/

## database.py

In [1]:
#%env POSTGRES_USERNAME=postgres
#%env POSTGRES_PASSWORD=#

import os
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

USERNAME: str = os.environ['POSTGRES_USERNAME']
PASSWORD: str = os.environ['POSTGRES_PASSWORD']
HOST: str = 'localhost'
PORT: int = 5432

engine = create_engine(f'postgresql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/postgres')

In [2]:
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

engine.connect()

In [15]:
import pandas as pd

#engine.execute("DROP TABLE transactions;")
df = pd.read_csv('../data/blacklist_results/haircut.csv', chunksize=100000, usecols=['address', 'balance', 'taint'])
for chunk in df:
    chunk['balance'] = chunk.balance.apply(lambda x: int(x) / 10**18)
    chunk.to_sql('haircut', engine, if_exists='append', index=False)
engine.execute('SELECT * FROM haircut;').fetchmany(3)

[('0xd4b88df4d29f5cedd6857912842cff3b20c8cfa3', 10000000.0, 1),
 ('0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144', 10000000.0, 1),
 ('0x07687e702b410fa43f4cb4af7fa097918ffd2730', 9999999.988587502, 1)]

## models.py

In [4]:
from sqlalchemy import Column, Integer, String, Float, DateTime, ForeignKey
from sqlalchemy.orm import relationship

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True, index=True)
    name = Column(String(50), nullable=False)
    email = Column(String(120), unique=True, nullable=False)

    def __repr__(self):
        return f'<User {self.name}>'

class Item(Base):
    __tablename__ = 'items'

    id = Column(Integer, primary_key=True, index=True)
    name = Column(String(50), nullable=False)
    description = Column(String(250))
    price = Column(Float, nullable=False)

    def __repr__(self):
        return f'<Item {self.name}>'